<a href="https://colab.research.google.com/github/markandrewsoliman/sched-aut/blob/main/scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from datetime import datetime
import asyncio
import nest_asyncio
import time
from telethon import TelegramClient, events
from telethon.errors import SessionPasswordNeededError
import sqlite3

nest_asyncio.apply()

API_ID = 26203352
API_HASH = 'ca2be12d169f7817cbc01a5f3ba580af'

SOURCE_GROUP_ID = -1001739812358
MESSAGE_ID = 32

DESTINATION_SETS = {
    1: [-1001843263657, -1002238665118, -1001358229200, -1888818526, -1001391721734, -1001435271098, -1001358229200, -1001391721734, -1001435271098, -1001270456524, -1001473487521, -1001431981808, -1001170418299, -1001289125640, -1001494427016, -1001493127931, -1001433496581, -1001358186422, -1001465794190, -1001354615413, -1001225635285, -1001085943722, -1001255210585, -1001555587234, -1001378145953, -1001319390715, -1001462365230, -1001729611504, -1001159126287, -1001379273366, -1001358661156, -1001379332226, -1001227368245, -1001245550964, -1001311790526, -1001405254454, -1001379420936, -1001259454821, -1001305830922, -1001428700086, -1001180218073, -1001205196662, -1001332205503, -1001261114547, -1001430405102, -1001245010351, -1001329950835, -1001412967174, -1001437684288, -1001383429967, -1001388029809, -1001207248876, -1001400604725, -1001302782255, -1001476510643, -1001396634313, -1001331135086, -1001471852664, -1001444483723, -1001495876403, -1001266787943],
    2: [-1001457682317, -1001238356209, -1888818526, -1001486934103, -1001260391401, -1001412489069, -1001455111422, -1001214804837, -1001459598289, -1001254354410, -1001413845967, -1001497220926, -1001198131718, -1001545621104, -1001417459096, -1001495514432, -1001236388750, -1001346068933, -1001401826587, -1001418967048, -1001251761464, -1001426782404, -1001331657499, -1001168540949, -1001431366082, -1001261110174, -1001488257051, -1001589467597, -1001186820460, -1001403559993, -1001151823042, -1001494942993, -1001431981808, -1001327361438, -1001495188348, -1001264374346, -1001329926109, -1001845372210, -1001404312159, -1001439097256, -1001458418279, -1001317985283, -1001463531438, -1001230418771, -1001254157253, -1001219877201, -1001460536366, -1001512534731, -1001459593094, -1001329917277, -1001265846172, -1001352981838, -1001448994516, -1001381313047, -1001235289327, -1001169087800, -1001264967245, -1001239379486, -1001592607417, -1001531480831, -1001305285907, -1001458881541],
    3: [-1001190267572, -1888818526, -1001667951540, -1001462685825, -1001321698163, -1001490948011, -1001202344162, -1001189951567, -1001318466170, -1001164867975, -1001321741450, -1001481051857, -1001421718699, -1001267886027, -1001285934353, -1001206257821, -1001440769348, -1001735054352, -1001564841852, -1001577668835, -1001744860430, -1001266742232, -1001301093493, -1001488152634, -1001467907325, -1001137161275, -1001481453225, -1001192638217, -1001228219840, -1001267489377, -1001394889403, -1001240563098, -1001195847383, -1001454526869, -1001491143690, -1001289952964, -1001341539429, -1001326058399, -1001254025617, -1001254285481, -1001317145992, -1001172881766, -1001375787941, -1001475006158, -1001301956329, -1001690609521, -1001165794313, -1001174495534, -1001739812358, -1001144996937],
    4: [-633537570, -1888818526]
}

# Define schedules for each set
SCHEDULES = {
    1: ["06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00"],
    2: ["06:20", "07:20", "08:20", "09:20", "10:20", "11:20", "12:20", "13:20", "14:20", "15:20", "16:20", "17:20", "18:20", "19:20", "20:20", "21:20", "22:20"],
    3: ["06:40", "07:40", "08:40", "09:40", "10:40", "11:40", "12:40", "13:40", "14:40", "15:40", "16:40", "17:40", "18:40", "19:40", "20:40", "21:40", "22:40"],
    4: ["06:50", "07:50", "08:50", "09:50", "10:50", "11:50", "12:50", "13:50", "14:50", "15:50", "16:50", "17:50", "18:50", "19:50", "20:50", "21:50", "22:50"]
}

SESSION_NAME = f'auto_forward_sched_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
client = TelegramClient(SESSION_NAME, API_ID, API_HASH)


async def forward_message_to_destination(message, destination_set):
    print(f"Forwarding message to set {destination_set}...")
    destination_group_ids = DESTINATION_SETS[destination_set]
    for destination_group_id in destination_group_ids:
        try:
            entity = await get_entity_with_retry(client, destination_group_id)
            await client.send_message(entity, message)
            print(f"Message forwarded to group {destination_group_id}: {message.text}")
        except Exception as e:
            print(f"Failed to forward message to group {destination_group_id}: {e}")

async def forward_specific_message(set_id):
    print("Fetching specific message...")
    message = await client.get_messages(SOURCE_GROUP_ID, ids=MESSAGE_ID)
    if message:
        await forward_message_to_destination(message, set_id)
    else:
        print(f"Message with ID {MESSAGE_ID} not found.")

async def schedule_forwarding():
    now = datetime.now()
    current_time = now.strftime("%H:%M")
    for set_id, start_times in SCHEDULES.items():
        if current_time in start_times:
            print(f"Scheduled time reached for set {set_id}: {current_time}")
            await forward_specific_message(set_id)
        else:
            print(f"Not yet time for set {set_id}. Current time: {current_time}")

async def get_entity_with_retry(client, group_id, retries=5):
    for attempt in range(retries):
        try:
            return await client.get_entity(group_id)
        except sqlite3.OperationalError:
            print("Database locked, retrying...")
            time.sleep(1)  # Wait before retrying
    raise Exception("Failed to get entity after several attempts")

async def main():
    print("Starting Telethon client...")
    await client.start()
    print("Client connected and authorized.")
    await schedule_forwarding()

async def get_entity_with_retry(client, group_id, retries=5):
    for attempt in range(retries):
        try:
            return await client.get_entity(group_id)
        except sqlite3.OperationalError:
            print("Database locked, retrying...")
            time.sleep(1)  # Wait before retrying
    raise Exception("Failed to get entity after several attempts")

# Run the main coroutine directly
await main()

Starting Telethon client...
Please enter your phone (or bot token): +639454226026
Please enter the code you received: 10738
Signed in successfully as read bio • for market plugging only; remember to not break the ToS or you will risk an account ban!
Client connected and authorized.
Not yet time for set 1. Current time: 20:18
Not yet time for set 2. Current time: 20:18
Not yet time for set 3. Current time: 20:18
Not yet time for set 4. Current time: 20:18
